## Imports

In [1]:
import ipywidgets as widgets
from IPython.display import display
import cv2
import io
import boto3
from matplotlib import pyplot as plt

## Define Global Variables

In [2]:
client = boto3.client('s3')
bucket='obstacles-classification'
datasets = []

# Get all dataset names from the s3 bucket
for prefix in client.list_objects(Bucket=bucket, Delimiter='/')['CommonPrefixes']:
    datasets.append(prefix['Prefix'][:-1])

## Define Widgets

In [3]:
dataset_dd = widgets.Dropdown(options=['<None>'] + datasets,
                              description='Dataset:',
                              value='<None>',
                              disabled=False,)
tog = widgets.ToggleButtons(options=['< prev', 'Obstacle', 'No Obstacle', 'Remove', 'next >'])
im = widgets.Image()
upload_button = widgets.Button(description='Upload Annotations')
upload_button.style.button_color = 'moccasin'
confirm_button = widgets.Button(description='Confirm All Annotations')
confirm_button.style.button_color = 'lightgreen'
output = widgets.Output()

## Widget Callback Functions

In [4]:
# Variables used by Callback Functions
image_names = []
cur_image = [0]
annotations = {}
tog_values = {
    'classified_obstacle': 'Obstacle',
    'classified_no_obstacle': 'No Obstacle'
}
tog_to_dst_name = {'Obstacle': 'train/obstacle',
                  'No Obstacle': 'train/no_obstacle'}

# Direct callback functions
def select_dataset(change):
    global selected_dataset
    selected_dataset = change['new']
    get_dataset_image_names(selected_dataset)

def tog_fn(change):
    handle_tog_fn(change['old'], change['new'])

# Utility functions used by callback functions
def get_dataset_image_names(dataset):
    for image_name in client.list_objects(Bucket=bucket, Prefix=dataset)['Contents']:
        if image_name['Key'].split('.')[-1] == 'jpg':
            image_names.append(image_name['Key'])
    display_image(image_names[cur_image[0]])
    annotation = tog_values[image_names[cur_image[0]].split('/')[1]]
    annotations.setdefault(cur_image[0], {'from': annotation})
    tog.value = annotation
    with output:
        print(f'current image: {cur_image[0]}')
        
def display_image(imname):
    client = boto3.client("s3")
    bucket = 'obstacles-classification'
    key = imname
    outfile = io.BytesIO()
    client.download_fileobj(bucket, key, outfile)
    outfile.seek(0)
    bytes_data = outfile.read()
    im.value = bytes_data
    
def handle_tog_fn(old, new):
    tog_button_functions[new](old, new)
    
def prev_image(old, new):
    cur_image[0] = max(cur_image[0]-1, 0)
    display_image(image_names[cur_image[0]])
    changed = {key: value for (key, value) in annotations.items() if value['from'] != value['to']}
    if cur_image[0] in changed.keys():
        annotation = changed[cur_image[0]]['to']
    else:
        annotation = tog_values[image_names[cur_image[0]].split('/')[1]]
        annotations.setdefault(cur_image[0], {'from': annotation})
    tog.value = annotation
    with output:
        print(f'current image: {cur_image[0]}')
            
def next_image(old, new):
    cur_image[0] = min(cur_image[0]+1, len(image_names))
    display_image(image_names[cur_image[0]])
    changed = {key: value for (key, value) in annotations.items() if value['from'] != value['to']}
    if cur_image[0] in changed.keys():
        annotation = changed[cur_image[0]]['to']
    else:
        annotation = tog_values[image_names[cur_image[0]].split('/')[1]]
        annotations.setdefault(cur_image[0], {'from': annotation})
    tog.value = annotation
    with output:
        print(f'current image: {cur_image[0]}')
    
def annotate(old, new):
    annotations[cur_image[0]]['to'] = new
    
def upload_annotations(change):
    changed = {key: value for (key, value) in annotations.items() if value['from'] != value['to']}
    with output:
        print(f'changed: {changed}')
    move_images(changed, 'No Obstacle', 'Obstacle')
    move_images(changed, 'Obstacle', 'No Obstacle')
    
def move_images(changed, src, dst):
    # Move all images changed from source to destination accordingly
    im_nums = {key: value for (key, value) in changed.items()\
           if value['from'] == src and value['to'] == dst}.keys()
    ims = [image_names[i] for i in im_nums]
    s3_resource = boto3.resource('s3')

    with output:
        print(f'ims: {ims}')
            
    for im in ims:
        subs = im.split('/')
        subs[1] = tog_to_dst_name[dst]
        new_path = '/'.join(subs)
        
        # Copy object A as object B
        s3_resource.Object(bucket, new_path).copy_from(CopySource={'Bucket': bucket, 'Key': im})

        # Delete the former object A
        s3_resource.Object(bucket, im).delete() 
        
def confirm_all_annotations(change):
    s3_resource = boto3.resource('s3')
    s3_client = boto3.client('s3')
    bucket_obj = s3_resource.Bucket(bucket)
    with output:
        print(f'selected_dataset: {selected_dataset}')
        
    # Copy all items left in 'classified_obstacle' to 'obstacle_pairs'
    
    src_prefix = selected_dataset + '/classified_obstacle/'
        
    for obj in bucket_obj.objects.filter(Prefix=src_prefix):
        imname = obj.key.split('/')[-1]
        copy_source = {'Bucket': bucket, 'Key': obj.key}
        src_obj = bucket_obj.Object(obj.key)
        dst_key = selected_dataset + '/train/obstacle/' + imname
        dst_obj = bucket_obj.Object(dst_key)
        with output:
            print(f'copy_source: {copy_source}')
            print(f'dst_key: {dst_key}')
        dst_obj.copy(copy_source)
        src_obj.delete()
        folder_key = selected_dataset + '/classified_obstacle/'
        s3_client.put_object(Bucket=bucket, Key=folder_key)

    # Copy all items left in 'classified_no_obstacle' to 'no_obstacle_pairs'
    
    src_prefix = selected_dataset + '/classified_no_obstacle/'
        
    for obj in bucket_obj.objects.filter(Prefix=src_prefix):
        imname = obj.key.split('/')[-1]
        copy_source = {'Bucket': bucket, 'Key': obj.key}
        src_obj = bucket_obj.Object(obj.key)
        dst_key = selected_dataset + '/train/no_obstacle/' + imname
        dst_obj = bucket_obj.Object(dst_key)
        with output:
            print(f'copy_source: {copy_source}')
            print(f'dst_key: {dst_key}')
        dst_obj.copy(copy_source)
        src_obj.delete()
        folder_key = selected_dataset + '/classified_no_obstacle/'
        s3_client.put_object(Bucket=bucket, Key=folder_key)

# Dictionary used by toggle buttons handler
tog_button_functions = {
    '< prev': prev_image,
    'Obstacle': annotate,
    'No Obstacle': annotate,
    'Remove': annotate,
    'next >': next_image
}

## Define & Display Widgets

In [5]:
dataset_dd.observe(select_dataset, names='value')
tog.observe(tog_fn, names='value')
upload_button.on_click(upload_annotations)
confirm_button.on_click(confirm_all_annotations)

display(dataset_dd, tog, im, upload_button, confirm_button, output)

Dropdown(description='Dataset:', options=('<None>', '', '3_class', 'debug_images', 'deeper_nn_experiment', 'ge…

ToggleButtons(options=('< prev', 'Obstacle', 'No Obstacle', 'Remove', 'next >'), value='< prev')

Image(value=b'')

Button(description='Upload Annotations', style=ButtonStyle(button_color='moccasin'))

Button(description='Confirm All Annotations', style=ButtonStyle(button_color='lightgreen'))

Output()

In [6]:
bucket

'obstacles-classification'

In [7]:
tog_values

{'classified_obstacle': 'Obstacle', 'classified_no_obstacle': 'No Obstacle'}

In [8]:
image_names

[]